In [3]:
import argparse
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
import gc
import argparse
import pandas as pd
import numpy as np
import re
from scipy import sparse
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
import fasttext
import fasttext.util
from transliterate import translit
import joblib

In [4]:
def parse_word_count(s: str) -> dict:
    """
    Parse a 'word_count' string into a dictionary of {word: count}.
    Performs cleaning (lowercasing, trimming punctuation) for robustness.
    """
    word_dict = {}
    if not isinstance(s, str):
        return word_dict  # return empty for non-string input
    
    # Split by whitespace and parse in pairs
    tokens = s.strip().split()
    if not tokens:
        return word_dict
    # Ensure even number of tokens (word, count pairs)
    if len(tokens) % 2 != 0:
        tokens = tokens[:-1]  # drop any odd trailing token
    
    # Iterate over word/count pairs
    for i in range(0, len(tokens), 2):
        word = tokens[i]
        count_str = tokens[i+1] if i+1 < len(tokens) else "0"
        # Clean the word: lowercase and strip punctuation at edges
        word_clean = word.lower()
        word_clean = re.sub(r'^[\W_]+|[\W_]+$', '', word_clean)  # remove non-alphanumeric underscores at ends
        if word_clean == "" or word_clean.isdigit():
            # Skip purely numeric or empty tokens (considered noise or already handled as counts)
            continue
        try:
            count = int(count_str)
        except:
            # If count is not a valid integer, skip this pair
            continue
        if count <= 0:
            continue
        # Accumulate counts for the word (merge duplicates if any)
        if word_clean in word_dict:
            word_dict[word_clean] += count
        else:
            word_dict[word_clean] = count
    return word_dict

In [35]:
# 1) Читаем TSV
df = #joblib.load('ptext_v1.joblib')


In [43]:
indices    = df['index'].values

In [49]:
joblib.dump(indices, 'text_indices.joblib')

['text_indices.joblib']

In [16]:
import gc
def dict_generator():
    for s in df['word_count'].fillna(''):
        yield parse_word_count(s)

In [17]:
import numpy as np
gc.collect()
# 3) DictVectorizer → матрица счётчиков (sparse)
dv = DictVectorizer(sparse=True,dtype=np.int32)
X_counts = dv.fit_transform(dict_generator())

print(X_counts.shape)


(274446, 64993)


In [22]:
#feature_names = dv.get_feature_names_out()
#np.save("feature_names",feature_names)

In [29]:
#sparse.save_npz('X_counts.npz', X_counts)

In [30]:
import gc
del df
gc.collect()

# 4) TF–IDF
tfidf = TfidfTransformer(norm=None, use_idf=True) #не нормируем тк нормируем в конце
X_tfidf = tfidf.fit_transform(X_counts)    # shape = [n_docs, V]

gc.collect()

0

In [31]:
from scipy import sparse

# Сохраняем в .npz формате
#sparse.save_npz('X_tfidf.npz', X_tfidf)

In [32]:
tfidf = TfidfTransformer(use_idf=True) #не нормируем тк нормируем в конце
X_tfidf = tfidf.fit_transform(X_counts)    # shape = [n_docs, V]

In [33]:
#sparse.save_npz('X_tfidfnorm.npz', X_tfidf)

In [53]:
X_tfidf = sparse.load_npz('X_tfidf.npz')

In [54]:
print(X_tfidf.shape)

(274446, 64993)


In [55]:
# 1) Загрузка предобученных бинарных моделей (они 300‑мерные)
en_model = fasttext.load_model('cc.en.300.bin')
ru_model = fasttext.load_model('cc.ru.300.bin')

In [56]:
# 2) Загружаем матрицы выравнивания https://github.com/babylonhealth/fastText_multilingual
A_en = np.loadtxt('alignment_matrices/en.txt')  # shape [300,300]
A_ru = np.loadtxt('alignment_matrices/ru.txt')

In [57]:
# 2) Теперь для каждого слова — только одна проекция
all_words  = np.load('feature_names.npy',allow_pickle=True) #dv.get_feature_names_out() #np.load('fature_names123.npy',allow_pickle=True)
V = len(all_words)
all_reduced = np.empty((V, 300))

en_vocab = set(en_model.words)
for i, w in enumerate(all_words):
    # 300‑мерный fastText‑вектор
    if w in en_vocab:
        v300 = en_model.get_word_vector(w)
        all_reduced[i] = v300.dot(A_en)
    else:
        v300 = ru_model.get_word_vector(w)
        all_reduced[i] = v300.dot(A_ru)

In [58]:

# 4) Собираем «словарь» векторов для всех фичей сразу (опционально для ускорения)
E = all_reduced

del all_reduced, en_model, ru_model
gc.collect()

289

In [59]:
print(E.shape)

(64993, 300)


In [60]:
from sklearn.preprocessing import normalize
doc_emb = X_tfidf.dot(E)
doc_emb = normalize(doc_emb, norm='l2', axis=1)

In [61]:
print(doc_emb.shape)
print(X_tfidf.shape)

(274446, 300)
(274446, 64993)


In [62]:
import joblib
col_names = [f'emb_{i}' for i in range(doc_emb.shape[1])]
indices = joblib.load('text_indices.joblib') 
df_emb = pd.DataFrame(doc_emb, index=indices, columns=col_names)
print(df_emb)
# 4) Сохраняем в CSV (можно добавить compression='gzip' при необходимости)
df_emb.to_csv('doc_emb_norm_2.csv')

           emb_0     emb_1     emb_2     emb_3     emb_4     emb_5     emb_6  \
81619   0.026933 -0.002485 -0.052109 -0.027761  0.032831  0.058368 -0.004885   
288108 -0.002521  0.031942 -0.024314  0.038365  0.041401  0.083446  0.024981   
252671 -0.015113 -0.022096 -0.038125 -0.017852  0.035653  0.026642  0.033787   
123476 -0.012253 -0.086710 -0.004167 -0.021619  0.025792  0.032874 -0.052965   
13202   0.020410  0.024960 -0.019017 -0.000660  0.027122  0.071049  0.000223   
...          ...       ...       ...       ...       ...       ...       ...   
24979   0.006815  0.057387 -0.075756 -0.012949  0.008710  0.023851  0.039128   
336266 -0.025174  0.032423 -0.047796 -0.037746  0.013046  0.017145  0.030569   
343454  0.018964 -0.018258 -0.051779 -0.068239  0.015676  0.000222 -0.020930   
123422 -0.000439  0.041426 -0.022043 -0.000849  0.008002  0.026170  0.008387   
23885  -0.005598  0.013536 -0.022707  0.059082  0.057551  0.042931  0.044471   

           emb_7     emb_8     emb_9  .

In [76]:

k = 300 # Количество признаков после svd(сингулярного разложения)
# 5) Truncated SVD → LSA‑пространство размерности k
svd = TruncatedSVD(n_components=k, random_state=42)
X_lsa = svd.fit_transform(X_tfidf)         # shape = [n_docs, k]

# 6) Собираем DataFrame с индексами и LSA‑фичами
lsa_cols = [f'lsa_{i+1}' for i in range(k)]
out_df = pd.DataFrame(X_lsa, columns=lsa_cols, index=indices)
out_df.index.name = 'index'
out_df.reset_index(inplace=True)

# 7) Сохраняем результат
out_df.to_csv("text_data_lsa300.csv", index=False)
print(f"LSA-признаки сохранены в text_data_lsa.csv")

LSA-признаки сохранены в text_data_lsa.csv


In [77]:

k = 200 # Количество признаков после svd(сингулярного разложения)
# 5) Truncated SVD → LSA‑пространство размерности k
svd = TruncatedSVD(n_components=k, random_state=42)
X_lsa = svd.fit_transform(X_tfidf)         # shape = [n_docs, k]

# 6) Собираем DataFrame с индексами и LSA‑фичами
lsa_cols = [f'lsa_{i+1}' for i in range(k)]
out_df = pd.DataFrame(X_lsa, columns=lsa_cols, index=indices)
out_df.index.name = 'index'
out_df.reset_index(inplace=True)

# 7) Сохраняем результат
out_df.to_csv("text_data_lsa200.csv", index=False)
print(f"LSA-признаки сохранены в text_data_lsa.csv")

LSA-признаки сохранены в text_data_lsa.csv
